# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [38]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san patricio,28.0170,-97.5169,87.75,49,59,13.65,US,1685225386
1,1,broken hill,-31.9500,141.4333,48.18,69,65,10.27,AU,1685225386
2,2,olonkinbyen,70.9221,-8.7187,31.01,72,54,15.88,SJ,1685225387
3,3,port-aux-francais,-49.3500,70.2167,41.47,87,100,32.91,TF,1685225387
4,4,at taj,24.2049,23.2857,74.98,36,0,17.81,LY,1685225387


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City"
)

# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] > 70) & (city_data_df['Max Temp'] < 81) & (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloudiness'] == 0)]

# ** Note: based on the VacationPy instructions, I used the conditions specicified. This file and WeatherPy are both showing in metrics in Imperial units so I converted the weather condition units based on that. **

# Drop any rows with null values -- There were no rows with null values base on these conditions
# ideal_city_df.info()
ideal_city_df.dropna(axis=0,how='any',inplace=False)

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,bandrele,-12.9067,45.1914,76.75,83,0,5.75,YT,1685225395
45,45,albany,42.6001,-73.9662,80.46,32,0,1.01,US,1685225398
99,99,thompson,55.7435,-97.8558,70.39,41,0,2.53,CA,1685225433
120,120,beaver dam,43.4578,-88.8373,76.21,21,0,8.05,US,1685225444
130,130,sarauli,28.5000,79.1000,77.29,58,0,3.89,IN,1685225453
151,151,remire-montjoly,4.9167,-52.2667,77.04,94,0,2.30,GF,1685225469
152,152,bethel,41.3712,-73.4140,78.40,41,0,8.05,US,1685225331
180,180,price,39.5994,-110.8107,74.12,18,0,6.91,US,1685225485
221,221,cullman,34.1748,-86.8436,79.11,53,0,6.91,US,1685225507
296,296,pisco,-13.7000,-76.2167,71.65,73,0,8.05,PE,1685225539


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
ideal_city_filter = ideal_city_df.copy()

hotel_df = ideal_city_filter[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
31,bandrele,YT,-12.9067,45.1914,83,
45,albany,US,42.6001,-73.9662,32,
99,thompson,CA,55.7435,-97.8558,41,
120,beaver dam,US,43.4578,-88.8373,21,
130,sarauli,IN,28.5000,79.1000,58,
151,remire-montjoly,GF,4.9167,-52.2667,94,
152,bethel,US,41.3712,-73.4140,41,
180,price,US,39.5994,-110.8107,18,
221,cullman,US,34.1748,-86.8436,53,
296,pisco,PE,-13.7000,-76.2167,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
limit = 20
radius = 10000

params = {
    "limit": limit,
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bandrele - nearest hotel: Les Baobabs
albany - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
beaver dam - nearest hotel: No hotel found
sarauli - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
bethel - nearest hotel: Hampton Inn Danbury
price - nearest hotel: Quality Inn Price Gateway To Moab National Parks
cullman - nearest hotel: Comfort Suites
pisco - nearest hotel: La Portada
sault ste. marie - nearest hotel: Holiday Inn
oneonta - nearest hotel: Courtyard by Marriott Oneonta
kone - nearest hotel: Pacifik Appartels
utraula - nearest hotel: No hotel found
jasper - nearest hotel: Econo Lodge
tamanrasset - nearest hotel: فندق الأمان
vila velha - nearest hotel: Hotel Prainha
broome - nearest hotel: No hotel found
waddan - nearest hotel: No hotel found
antibes - nearest hotel: Hôtel La Villa


,City,Country,Lat,Lng,Humidity,Hotel Name
31,bandrele,YT,-12.9067,45.1914,83,Les Baobabs
45,albany,US,42.6001,-73.9662,32,No hotel found
99,thompson,CA,55.7435,-97.8558,41,Thompson Inn
120,beaver dam,US,43.4578,-88.8373,21,No hotel found
130,sarauli,IN,28.5000,79.1000,58,No hotel found
151,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
152,bethel,US,41.3712,-73.4140,41,Hampton Inn Danbury
180,price,US,39.5994,-110.8107,18,Quality Inn Price Gateway To Moab National Parks
221,cullman,US,34.1748,-86.8436,53,Comfort Suites
296,pisco,PE,-13.7000,-76.2167,73,La Portada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE